# Fedbiomed Image Classifier with Differential Privacy with CIFAR10

In this tutorial we will show how an Image classifier with Opacus (https://opacus.ai/) differential privacy engine can be trained with Fed-BioMed. We refer to the Opacus tutorial available here:
https://opacus.ai/tutorials/building_image_classifier

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fedbiomed.researcher.requests import Requests
req = Requests()
req.list(verbose=True)

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default), and write CIFAR10 to add CIFAR to the node through `torchvision.datasets.CIFAR10`
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where CIFAR is downloaded
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a TorchTrainingPlan Net class to send for training on the node

In [ ]:
from fedbiomed.researcher.environ import environ
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=environ['TMP_DIR']+'/')
model_file = tmp_dir_model.name + '/Cifar_opacus.py'

In the cell below, we are going to define the model using opacus for differential privacy. For this example, we are going to use the `ModuleValidator` function to validate and/or correct models to be compatible with the `opacus` engine, and the function `make_private_with_epsilon` from `opacus.privacy_engine`. 

To train a model with `make_private_with_epsilon` from Opacus library, there are three privacy-specific hyper-parameters that must be tuned for better performance:

* `max_grad_norm`: The maximum L2 norm of per-sample gradients before they are aggregated by the averaging step.
* `noise_multiplier`: The amount of noise sampled and added to the average of the gradients in a batch.
* `target_epsilon` and `target_delta`: The target ϵ and δ of the (ϵ,δ)-differential privacy guarantee. 

It is worth noting that in order to use the opacus `PrivacyEngine` class we need to properly define as training plan attributes a `model`, a `dataloader` and an `optimizer`.

In [ ]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnnDP import TorchTrainingDPPlan
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, transforms, models
from opacus.validators import ModuleValidator
from opacus.utils.batch_memory_manager import BatchMemoryManager

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class CIFAR10DPPlan(TorchTrainingPlan):
    def __init__(self, model_args):
        super(CIFAR10DPPlan, self).__init__()
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms, models",
                "from torch.utils.data import DataLoader",
                "import torch.optim as optim",
                "from opacus.validators import ModuleValidator",
                "from opacus.utils.batch_memory_manager import BatchMemoryManager",]
        self.add_dependency(deps)
        
        self.diff_privacy = model_args['diff_privacy']
        self.privacy_func = model_args['privacy_func']
        self.max_grad_norm = model_args['max_grad_norm']
        self.epsilon = model_args['target_epsilon']
        self.delta = model_args['target_delta']
        self.max_physical_batch_size = model_args['max_physical_batch_size']
        
        self.num_classes = model_args['num_classes']
        self.model = self.make_model()
        self.loss = nn.CrossEntropyLoss()
        
    def make_model(self):
        model = models.resnet18(num_classes=self.num_classes)
        model = ModuleValidator.fix(model)
        ModuleValidator.validate(model, strict=False)      
        return model
    
    def give_optimizer(self,lr):
        optimizer = optim.RMSprop(self.model.parameters(), lr=lr)
        return optimizer
    
    def train_model(self):
        self.model.train()

    def forward(self, x):
        return self.model(x)

    def training_data(self, batch_size = 48):
        CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
        CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)
        # Custom torch Dataloader for CIFAR data
        transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
                                       ])
        dataset1 = datasets.CIFAR10(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = self.loss(output, target)
        return loss

This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side. For instance, the privacy parameters should be passed here.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [ ]:
model_args = {'diff_privacy': True, 'privacy_func': 'make_private_with_epsilon',
              'num_classes': 10, 'max_grad_norm': 1.2, 
              'target_epsilon': 50.0, 'target_delta': 1e-5,
              'max_physical_batch_size': 100}

training_args = {
    'batch_size': 50, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

# Train the federated model

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#CIFAR10', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #nodes=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='CIFAR10DPPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the nodes

In [ ]:
exp.run()

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')

# Test Model

We define a little testing routine to extract the accuracy metrics on the testing dataset
## Important
This is done to test the model because it can be accessed in a developpement environment  
In production, the data wont be accessible on the nodes, need a test dataset on the server or accessible from the server.

In [ ]:

import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
import os

def testing_Accuracy(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    device = "cpu"

    correct = 0
    
    loader_size = len(data_loader)
    with torch.no_grad():
        for idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            #only uses 10% of the dataset, results are similar but faster
            if idx >= loader_size / 10:
                pass
                break

    
        pred = output.argmax(dim=1, keepdim=True)

    test_loss /= len(data_loader.dataset)
    accuracy = 100* correct/(data_loader.batch_size * idx)

    return(test_loss, accuracy)

Test dataset

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
import os

# These values, specific to the CIFAR10 dataset, are assumed to be known.
# If necessary, they can be computed with modest privacy budget.
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD_DEV),
])

base_dir = tmp_dir_model.name 
if not os.path.isdir(os.path.join(base_dir, "cifar10")):
    os.makedirs(os.path.join(base_dir, "cifar10"))
test_data_dir = os.path.join(base_dir, "cifar10")

test_dataset = CIFAR10(
    root=test_data_dir, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=48,
    shuffle=False,
)

We define a util function to calculate the accuracy:

In [ ]:
def accuracy(preds, labels):
    return (preds == labels).mean()

We define the model, and we assign to it the model parameters estimated at the last federated optimization round.

In [ ]:
from torchvision import models
from opacus.validators import ModuleValidator

model = models.resnet18(num_classes=10)
model = ModuleValidator.fix(model)
ModuleValidator.validate(model, strict=False)

model = exp.model_instance
model.load_state_dict(exp.aggregated_params[rounds - 1]['params'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

We define a function to validate our model on our test dataset.

In [ ]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)

And we finally test our model!

In [ ]:
top1_acc = test(model, test_loader, device)